In [317]:
from collections import Counter
import numpy as np
from threading import Thread
from multiprocessing import Process, Manager, freeze_support, set_start_method, Array, Queue
from time import time, sleep
import concurrent.futures
from helper2 import add_with_shared,add_without_shared
import multiprocessing, math
import pandas as pd

In [318]:
def add_vector_sequential(A,B):
    N = len(A)
    C = np.zeros(shape = (N))
    for i in range(N):
        C[i] = A[i]+B[i]
        #print(multiprocessing.current_process().name)
    return C

In [319]:
def add_parallel(A, B, C, start, end):
    for i in range(start,end):
        C[i] = A[i]+B[i]

def add_vector_threads(A,B,cores):
    N = len(A)
    steps = math.ceil(N//cores)#number of rows should be processed by each thread
    start = 0
    threads = []
    C = np.zeros(shape = (N))

    if __name__ == '__main__':
        while start<N:
            end = N if start+steps > N else start+steps
            t = Thread(target=add_parallel, args = [A,B,C,start,end])
            threads.append(t)
            start = end #update row number for the start of next process
        for t in threads: t.start()
        for t in threads: t.join()
    return C

In [320]:
def add_vector_multiprocess_shared_object(A,B,cores):
    N = len(A)
    #res = []
    steps = N//cores
    start = 0
    processes = []
    
    if __name__ == '__main__':
        manager = multiprocessing.Manager()
        return_dict = manager.dict()
        for c in range(cores):
            end = start+steps
            if cores-c==1:end = N
            p = Process(target=add_with_shared, args = [A[start:end],B[start:end],c,return_dict])
            processes.append(p)
            start = end
        for p in processes: p.start()
        for p in processes: p.join()
        res_list = []
        for c in range(cores):res_list+=return_dict[c]
        #for c in range(cores):res_list+=return_dict.get()
        #print(return_dict.values())
        return res_list

In [321]:
def add_vector_multiprocess_not_shared_object(A,B,cores):
    N = len(A)
    #res = []
    steps = N//cores
    start = 0
    processes = []
    
    if __name__ == '__main__':
        #manager = multiprocessing.Manager()
        #return_dict = manager.dict()
        C = [0]*N
        for c in range(cores):
            end = start+steps
            if cores-c==1:end = N
            p = Process(target=add_without_shared, args = [A[start:end],B[start:end],C[start:end]])
            processes.append(p)
            start = end
        for p in processes: p.start()
        for p in processes: p.join()

        #print(return_dict.values())
        return C

In [322]:
def compare_proc_sizes_for_N(N):
    procs = [1,2,3,4]
    df_compare_add_value = pd.DataFrame(columns=['Process count','Sequential','Multithreading','Multiprocessing with sharing objects','Multiprocessing without sharing objects'])
    A = np.random.random(size = (N))
    B = np.random.random(size = (N))
    for i in range(len(procs)):
        proc = procs[i]
        t0 = time()
        C = add_vector_sequential(A,B)
        t1 = time()
        #print(C)
    
        D = add_vector_threads(A,B,proc)
        t2 = time()

        E = add_vector_multiprocess_shared_object(A,B,proc)
        t3 = time()
        #print(E)
        
        F = add_vector_multiprocess_not_shared_object(A,B,proc)
        t4 = time()
        #print(E)
        df_compare_add_value.loc[i] = [proc, t1-t0, t2-t1, t3-t2, t4-t3]
    return df_compare_add_value


In [324]:
df_pow_4 = compare_proc_sizes_for_N(pow(10,4))
df_pow_4

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects,Multiprocessing without sharing objects
0,1.0,0.008195,0.008854,0.722440,0.325491
1,2.0,0.006120,0.009950,0.713550,0.401473
2,3.0,0.006147,0.007402,0.746443,0.393401
3,4.0,0.005430,0.006483,0.873475,0.463086


In [325]:
df_pow_5 = compare_proc_sizes_for_N(pow(10,5))
df_pow_5

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects,Multiprocessing without sharing objects
0,1.0,0.095905,0.057628,1.697225,0.313860
1,2.0,0.061037,0.062107,1.335632,0.504758
2,3.0,0.055495,0.056365,1.509320,0.800274
3,4.0,0.056159,0.060533,1.663648,0.930309


In [326]:
df_pow_6 = compare_proc_sizes_for_N(pow(10,6))
df_pow_6

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects,Multiprocessing without sharing objects
0,1.0,0.585214,0.561351,11.082020,0.899464
1,2.0,0.561581,0.566540,8.379571,0.948914
2,3.0,0.597298,0.601421,8.189399,1.178641
3,4.0,0.580389,0.710423,9.070524,1.354424


In [327]:
df_pow_7 = compare_proc_sizes_for_N(pow(10,7))
df_pow_7

,Process count,Sequential,Multithreading,Multiprocessing with sharing objects,Multiprocessing without sharing objects
0,1.0,6.120191,5.990518,165.929810,8.276228
1,2.0,5.928665,6.032258,153.840145,5.888988
2,3.0,6.074759,5.977701,128.824786,5.459659
3,4.0,5.915720,5.892937,104.201140,6.372270


In [328]:
df_pow_8 = compare_proc_sizes_for_N(pow(10,8))
df_pow_8

KeyboardInterrupt: 